In [ ]:
from pathlib import Path

import happy_wheel
from happy_wheel import DATA_ROOT
from happy_wheel.config import TrainConfig, ModelConfig, LossConfig, OptimizerConfig, DatasetConfig, Config
import ykaggle_core as ycore

ycore.set_gpu(0)

In [ ]:
train_config = TrainConfig(epochs=30)
model_config = ModelConfig(
    name="efficientnet-b0",
    kwargs=dict(
        include_top=False,
        input_shape=(image_height, image_width, 3),
    ),
)

train_dataset_config = DatasetConfig(
    input_dir=DATA_ROOT / "preprocessed" / "train_images",
    batch_size=32,
    width=image_width,
    height=image_height,
    shuffle=True,
    label_csv_path=DATA_ROOT / "preprocessed" / f"train_cv{fold}.csv",
)
val_dataset_config = DatasetConfig(
    input_dir=DATA_ROOT / "preprocessed" / "train_images",
    batch_size=64,
    width=image_width,
    height=image_height,
    shuffle=False,
    label_csv_path=DATA_ROOT / "preprocessed" / f"valid_cv{fold}.csv",
)
config = Config(
    exp_name=f"{base_exp_name}/cv{fold}",
    train=train_config,
    model=model_config,
    loss=LossConfig(name="categorical_crossentropy"),
    optimizer=OptimizerConfig(name="adam"),
    train_dataset=train_dataset_config,
    validation_dataset=val_dataset_config,
    test_dataset=val_dataset_config,
)